In [1]:
import pandas as pd
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.models import BasicTickFormatter, Label, Title, NumeralTickFormatter
# HoverTool allows you to set a tooltips property which takes a list of tuples
from bokeh.models.tools import HoverTool
# a pre-made five color pallette, one of Bokeh’s many pre-made color palettes
from bokeh.palettes import Spectral6
# a helper method for mapping colors to bars in a bar-charts
from bokeh.transform import factor_cmap
import math

datapath = 'datapandas/'
inputFile1 = 'ABS2021StateAgegrpIntPopulation.xlsx'     # population count by State, Age, AgeGroup
inputFile2 = 'ReferencePopulationGenerationGroup.xlsx'  # contains age group description

# Function "output_file" defines the output name and how the visualization will be rendered by bokeh, eg html for this one.
# File will be saved in the main project folder
output_file('codeflask/templates/populationByGeneration.html')

# Read the two input excel files
dfPopAgeByGrp = pd.read_excel(datapath+inputFile1)
#print(dfPopAgeByGrp,'\n')
dfGenAgeGrp = pd.read_excel(datapath+inputFile2)
#print(dfGenAgeGrp,'\n')

# Group population data by AgeGroup
dfGrouped = dfPopAgeByGrp.groupby('AgeGroup')[['Population']].sum()
#print(dfGrouped,'\n')

# Compute percentage of population for each age group
dfGrouped['percent'] = round((dfGrouped['Population'] / dfGrouped['Population'].sum()),4) * 100
#print(dfGrouped,'\n')

# Merge population data with generation description
dfMerged = pd.merge(dfGrouped,dfGenAgeGrp,how='left',on='AgeGroup')
#print(dfMerged,'\n')
#            AgeGroup  Population  percent   Description
#0          0-9 years     3049948    12.00     Gen Alpha
#1        10-24 years     4625406    18.19         Gen Z
#2        25-39 years     5463609    21.49   Millennials
#3        40-54 years     4895725    19.26         Gen X
#4        55-74 years     5469220    21.51  Baby Boomers
#5  75 years and over     1918837     7.55      Interwar

# Round percentage again
dfMerged['percent'] = round(dfMerged['percent'],4)
#print(dfMerged,'\n')

# Add percentage value to the description
dfMerged['Description']=dfMerged['Description'] + ": " + dfMerged['percent'].apply(str) + "%"
#print(dfMerged,'\n')

# Set dataframe into 'column data source' so it can be processed by bokeh
source = ColumnDataSource(dfMerged)
# Get list of Categorical 'Description' values for vertical bar chart (x-axis)
ageDesc = source.data['Description'].tolist()
#print(ageDesc,'\n')
#['Gen Alpha: 12.0%', 'Gen Z: 18.19%', 'Millennials: 21.49%', 'Gen X: 19.26%', 'Baby Boomers: 21.51%', 'Interwar: 7.55%']

#--------------------------------------------------------------------PLOT SIZE
# Create 'figure' object, the core object that handles the styling of plots in bokeh.
# Pass ageDesc to x_range in the 'figure' constructor
#p = figure(x_range=ageDesc, sizing_mode="stretch_both") # sizing_mode=stretch_width   sizing_mode=stretch_height
p = figure(x_range=ageDesc, width=700, height=400
           , background_fill_color="#292929" #404040
           , border_fill_color = "black"
           , outline_line_color="white"
           , outline_line_width = 0
           #, outline_line_alpha = 0.3
           )

# Plot data as individually colored bars and add basic labels.
# To color the bars we use the factor_cmap helper function.
# This creates a special color map that matches an individual color to each Category (ageDesc)
color_map = factor_cmap(field_name='Description', # create a special color map that matches an individual color to each category
                        palette=Spectral6,
                        factors=ageDesc)      # factors in bokeh refers to Categories.
# Call vbar method to create a vertical bar Glyph
p.vbar(x='Description', top='Population', source=source, width=0.50, color=color_map)
# Add labels
p.title.text ='Population By Generation as at 2021 Census'
p.xaxis.axis_label = 'Generation'
p.yaxis.axis_label = 'Population'
#------------------------------------------------------------------------
# Specify Axis labels
##axis_label_text_color = 'red'
##axis_line_color = 'red'
p.yaxis.major_label_text_color = "#e0e0e0"        #ok
p.yaxis.axis_label_text_color = "#e0e0e0"         #ok
p.xaxis.major_label_text_color = "#e0e0e0"        #ok
p.yaxis.major_label_text_font_size = "9pt"
p.xaxis.major_label_text_font_size = "8pt"
p.xaxis.axis_label_text_color = "#e0e0e0"         #ok

# Change orientation of x-axis labels
#p.xaxis.major_label_orientation = math.pi/3
#p.xaxis.major_label_orientation = "vertical"

# Change scientific notation in y-axis
p.yaxis[0].formatter = NumeralTickFormatter(format="0,0")

# Chart title
p.title.text_color = "#e0e0e0"
#Hide Gridlines
p.xgrid.grid_line_color = "#e0e0e0"
p.ygrid.grid_line_color = "#e0e0e0"
#--------------------------------------------------------------------------
# Add hover
hover = HoverTool()    # HoverTool allows you to set a tooltips property which takes a list of tuples
hover.tooltips = [("","@Description;  Age:@AgeGroup;  Population:@Population{0,0}")]
hover.mode = 'vline'   # tell the popup to show when a vertical line crosses a glyph
p.add_tools(hover)     # add hover tool in the toolbar.

#-----------------------------------------------------
#labels = LabelSet(x='ageDesc', y='Population', text='percent', x_offset=50, y_offset=10, source=source)
#p.add_layout(labels)
#-----------------------------------------------------

show(p)
#save(p)


#end of file